In [2]:
import urllib
import xml.etree.ElementTree as ET
import os
import wget
import time
import numpy as np
import pandas as pd
import seaborn as sns
import re
import tarfile as tf

try:
    os.mkdir('VPA')
except: #if file already exists, do nothing and move to next command
    pass
try:
    os.mkdir('VPA/MINIML')
except:
    pass
try:
    os.mkdir('VPA/Datasets')
except:
    pass
try:
    os.mkdir('VPA/GSM_Tables')
except:
    pass
try:
    os.mkdir('VPA/Info')
except:
    pass


In [3]:
# downloading MINIML format

missingFileList=[]
output_fileName='VPA/Info/vpa_f1_extractMINIML_missingFiles.txt'
ds=pd.read_csv('VPA/Info/vpa_search_f1.csv')
for i in ds['DataLink']:
    try:
        wget.download(i,'VPA/MINIML')
        time.sleep(0.34)
    except:
        missingFileList.append(i)

if missingFileList!=[]: #if there are missing files
    output_fileObject=open(output_fileName, "w") #create new text file
    for i in missingFileList:
        outputStringToPrint=str(i)+"\n"
        output_fileObject.write(outputStringToPrint) 

output_fileObject.close()


100% [..............................................................] 13491826 / 13491826

In [4]:
# Extracting data to 'Datasets' folder

os.chdir('VPA/MINIML')

for file in os.listdir('./'):
    os.mkdir('VPA/Datasets/'+file)

for file in os.listdir('./'):
    try:
        tar=tf.open(file)
        tar.extractall(path='VPA/Datasets/'+file)
        tar.close
    except: continue


In [118]:
# Extracting GSM meta data from xml files

os.chdir('VPA/Datasets/')
dirs=os.listdir()

for ts in dirs:
    os.chdir('VPA/Datasets/'+ts+'/')
    lst=os.listdir()

    for i in lst:
        if i.find('xml')>-1:
            xml=i
        else: continue
    try: ids=ET.parse(xml).getroot()
    except: 
        misFile=open('VPA/GSM_Tables/'+ts+'.txt', 'w')
        misFile.write('There is no xml file, check other formats')
        misFile.close()
    
    # information about data processing
    prefix='{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}'
    gsm=list()
    gpl=list()
    procInf=list()
    gse=list()
    chrs=list()
    desc=list()
    columns=list()
    organism=list()
    source=list()
    
    # getting platform refernce number
    for i in ids.findall(prefix+'Sample/'+prefix+'Platform-Ref'):
        try: 
            platform=str(i.attrib.values())
            platform=re.findall("'.+'", platform)[0]
            gpl.append(platform)    
        except: gpl.append('check')    

    # getting ifor about data processing in the series
    for i in ids.findall(prefix+'Sample/'+prefix+'Data-Processing'):
        try: procInf.append(i.text.strip())    
        except: procInf.append('check')    

    # GSM accession number        
    for i in ids.findall(prefix+'Sample/'+prefix+'Accession'):
        try: gsm.append(i.text.strip())   
        except: gsm.append('check')
        gse.append(re.findall('GSE.+\d',xml)[0])

    # sample characteristics information    
    for i in ids.iter(prefix+'Sample'):
        chnl=list()
        for z in i.iter(prefix+'Channel'):
            chnl.append(z.tag)
        if len(chnl) == 1:
            chr=''
            for x in z.iter(prefix+'Characteristics'):
                try:
                    val=str(x.attrib.values()).strip()
                    val=re.findall("'.+'", val)[0]
                    chr=chr+val+' : '+x.text.strip()+',    '
                except: chr='Nothing here'
            chrs.append(chr)
            orgs=list()
            for t in z.iter(prefix+'Organism'):
                orgs.append(t.text)
            orgs=' + '.join(orgs)
            organism.append(orgs)
            for n in z.iter(prefix+'Source'):
                source.append(n.text)
        else: 
            chrs.append('Multiple channels: '+str(len(chnl)))
            organism.append('Multiple channels: '+str(len(chnl)))
            for n in z.iter(prefix+'Source'):
                source.append(n.text)
            
    # Sample description        
    for i in ids.iter(prefix+'Sample'):
        a='no data'
        des=i.findall(prefix+'Description')
        if len(des)>0:
            for z in des:
                desc.append(z.text.strip())
        else: 
            desc.append(a)

    # column headings in data files
    for i in ids.iter(prefix+'Sample'):
        for z in i.iter(prefix+'Data-Table'):
            colnms=''
            for x in z.findall(prefix+'Column/'+prefix+'Name'):
                colnms=colnms+'   '+x.text
        columns.append(colnms)

    sumFile=pd.DataFrame({'GSE':gse,'GPL':gpl,'GSM':gsm,'Organism':organism,'SampleInfo':chrs,
                     'SampleDesc':desc,'SampleSource':source,'ProcessingInfo':procInf,'ColNames':columns})
    sumFile.to_csv('VPA/GSM_Tables/'+xml+'.csv')
    

In [119]:
# combining meta-data for each GSM

os.chdir('VPA/GSM_Tables/')
gsm=os.listdir('.')

ds=pd.read_csv(gsm[0])
for i in range(len(gsm)-1):
    ind=i+1
    ds1=pd.read_csv(gsm[ind])
    ds=pd.concat([ds,ds1])

ds.to_csv('../Info/GSM-Meta_combined.csv')

In [ ]:
os.chdir('../')

In [ ]:
try:
    os.mkdir('VPA/GSM_Merged')
except:
    pass

In [ ]:
# Combining GSMs from all GSEs into one table


for gseSet in os.listdir('VPA/Datasets/'):
    print(gseSet)
    data=pd.DataFrame()
    z=pd.DataFrame()
    lst=os.listdir('VPA/Datasets/'+gseSet)
    for gpl in lst:
        if gpl.startswith('GPL'):
            #data=pd.read_table(gseSet+'/'+gpl,header=None)
            data=pd.read_table('VPA/Datasets/'+gseSet+'/'+gpl,header=None)
            data=data.rename(columns={0:'ID'})
        else: continue

    for i in lst:
        if i.startswith('GSM'):
            #z=pd.read_table(gseSet+'/'+i,header=None)
            z=pd.read_table('VPA/Datasets/'+gseSet+'/'+i,header=None)
            z=z.iloc[:,0:2]
            z.columns=['ID',i]
            try:
                data=pd.merge(data,z,on='ID',how='right')
                print("ok")
            except: continue
        else: continue

    data.to_csv('VPA/GSM_Merged/'+gseSet+'.csv')